# 3.3 plant development rate

In [ ]:
# sensitivity

In [ ]:
import pandas as pd
import statsmodels.api as sm
import os

In [ ]:
pep_dir = '/data/shuping/data/pep/'
out_dir = '/data/shuping/data/sensitivity/pep'
files = sorted(os.listdir(pep_dir))
for i in range(len(files)):
    species_dir = os.path.join(pep_dir, files[i])
    sensitivity = pd.DataFrame(columns=['PEP_ID', 'year', 
                                        'eos-GDDpre-r', 'eos-GDDpre-p_val',
                                        'eos-GDDpost-r', 'eos-GDDpost-p_val',
                                        ])
    species_files = sorted(os.listdir(species_dir))
    for k in range(len(species_files)):
        csv_path = os.path.join(species_dir, species_files[k])
        df = pd.read_csv(csv_path)
        df = df.dropna()
        df = df.drop(['Unnamed: 0'], axis=1)
        if df.shape[0]>15:
            start_year = df.iloc[0, 1]
            end_year = df.iloc[df.shape[0]-1, 1]
            if start_year + 14 <= end_year:
                for j in range(start_year, end_year-13):
                    data = df[(df['year'] < j +15) & (df['year'] >= j)]
                    X = data[['sos','pre_sum_tem', 'post_sum_tem', 'pre', 'solr', 'sm', 'gpp']]
                    X = sm.add_constant(X)
                    Y = data['eos']
                    model = sm.OLS(Y, X).fit()
                    sensitivity = sensitivity.append(pd.DataFrame({'PEP_ID':[df.PEP_ID[0]], 'year':[j+7],
                                                                   'eos-GDDpre-r':[model.params[2]],'eos-GDDpre-p_val': [model.pvalues[2]],
                                                                   'eos-GDDpost-r':[model.params[3]],'eos-GDDpost-p_val': [model.pvalues[3]],

                                                                   }), ignore_index=True)
    sensitivity.to_csv(os.path.join(out_dir, files[i] +'.csv'))